<a href="https://colab.research.google.com/github/venkatesh-r96/NLP-Assignment/blob/main/Venkatesh_NLP_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP ASSIGNMENT**

# OBJECTIVE


Automatically generate ojbective questions with multiple correct answers based on a given chapter from a subject.

In [ ]:
# Installing the required Packages
!pip install tiktoken

In [ ]:
!pip install sentence_transformers

In [ ]:
# For reading PDF files
!pip install PyPDF2

In [ ]:
# we are giving the solution through openai
!pip install openai

In [ ]:
# Finding the solution through google palm also
!pip install -q google-generativeai

In [ ]:
#Importing the required packages
import os
import nltk
import torch
from sentence_transformers import SentenceTransformer, util
import textwrap
import PyPDF2
import openai
openai.api_key = "XXXXXX" # Replace with your API key
import google.generativeai as palm
palm.configure(api_key ="XXXXXX") # Replace with your API key

In [ ]:
# Reading PDF file
with open('/content/chapter-2.pdf', 'rb') as file:
  pdf_reader = PyPDF2.PdfReader(file)
  content = ''
  for page in pdf_reader.pages:
    content += page.extract_text()

In [ ]:
# Checking content of the file
content

'From Trade to Territory                   \nThe Company Establishes Power2\nAurangzeb was the last of the powerful Mughal rulers. He \nestablished control over a very large part of the territory that is now known as India. After his death in 1707, many Mughal governors ( subadars) and big zamindars \nbegan asserting their authority and establishing regional kingdoms. As powerful regional kingdoms emerged in various parts of India, Delhi could no longer function as an effective centre.\nBy the second half of the eighteenth century, \nhowever, a new power was emerging on the political horizon – the British. Did you know that the British originally came as a small trading company and were reluctant to acquire territories? How then did they come to be masters of a vast empire? In this chapter you will see how this came about.\nFig. 1 – Bahadur Shah Zafar  and his sons being arrested by Captain Hodson\nAfter Aurangzeb there was no \npowerful Mughal ruler, but Mughal emperors continued to b

In [ ]:
# Checking token counts
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
tokens = tokenizer.encode(content)
print('Tok', tokens)

# Because of ChatGPT token limitations splitting the content into splits. A split contains token of 3000.
if len(tokens)>3000:
  tokens = tokenizer.encode(content)
  print('Len_Tokens', len(tokens))
  i=0
  n=3000
  a=[]
  while i < len(tokens):
    j=min(i+int(n), len(tokens))
    while j>i + int(n):
      # Decode the tokens and check for full stop or newline
      chunk = tokenizer.decode(tokens[i+j])
      if chunk.endswith(".") or chunk.endswith("\n"):
        break
      j -=1
    a.append(tokens[i:j])
    i = j


Tok [4863, 9601, 284, 25219, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 198, 464, 5834, 10062, 397, 19724, 4333, 17, 198, 32, 333, 648, 38130, 373, 262, 938, 286, 262, 3665, 337, 6724, 282, 24925, 13, 679, 220, 198, 27718, 1630, 625, 257, 845, 1588, 636, 286, 262, 7674, 326, 318, 783, 1900, 355, 3794, 13, 2293, 465, 1918, 287, 1596, 2998, 11, 867, 337, 6724, 282, 26824, 357, 850, 324, 945, 8, 290, 1263, 1976, 321, 521, 945, 220, 198, 1350, 1030, 33183, 511, 4934, 290, 15010, 7915, 41901, 13, 1081, 3665, 7915, 41901, 9349, 287, 2972, 3354, 286, 3794, 11, 12517, 714, 645, 2392, 2163, 355, 281, 4050, 7372, 13, 198, 3886, 262, 1218, 2063, 286, 262, 45592, 4289, 11, 220, 198, 4919, 964, 11, 257, 649, 1176, 373, 11823, 319, 262, 1964, 17810, 784, 262, 3517, 13, 7731, 345, 760, 326, 262, 3517, 6198, 1625, 355, 257, 1402, 7313, 1664, 290, 547, 18416, 284, 12831, 16771, 30, 1374, 788, 750, 484, 1282, 284, 307, 18159, 286, 257, 5909, 13735, 30,

In [ ]:
len(a)

3

Here len(a) is 3. The given content file has more than 3000 token. so it splitted the content to satisfy the given condition.(Exapmle: (8000=3000+3000+2000))

In [ ]:
# Decoding the token into text
text_chunks = [tokenizer.decode(chunk) for chunk in a]
text_chunks


['From Trade to Territory                   \nThe Company Establishes Power2\nAurangzeb was the last of the powerful Mughal rulers. He \nestablished control over a very large part of the territory that is now known as India. After his death in 1707, many Mughal governors ( subadars) and big zamindars \nbegan asserting their authority and establishing regional kingdoms. As powerful regional kingdoms emerged in various parts of India, Delhi could no longer function as an effective centre.\nBy the second half of the eighteenth century, \nhowever, a new power was emerging on the political horizon – the British. Did you know that the British originally came as a small trading company and were reluctant to acquire territories? How then did they come to be masters of a vast empire? In this chapter you will see how this came about.\nFig. 1 – Bahadur Shah Zafar  and his sons being arrested by Captain Hodson\nAfter Aurangzeb there was no \npowerful Mughal ruler, but Mughal emperors continued to 

# Using Openai by passing passage into gpt-3.5-turbo model, Generating mca questions

In [ ]:
# mca question generation funtion
def get_mca_questions(context):
    current_question = [{"role": "system", "content": """You are an AI assitant tasked with generating multiple-choice questions (MCQs) by reading the given context. Each MCQ should have 4 options in which 2 options must be the answer for that question and 2 options are unrelated to the question. Your goal is to ensure that the incorrect options appear plausible enough to challenge the test-takers' understanding and reasoning abilities. At the end of each question, please include a tag labeled 'Answers:' followed by the correct answer options for that particular question.
    Generate the Multiple Choice question as per Given Example format.
    Each questions must have two answers.
    Example:
    Question 1: Which of the following planets in our solar system have rings?
    A) Mercury
    B) Venus
    C) Saturn
    D) Mars
    Correct Answers: C) Saturn and D) Mars
    Multiple-choice-questions: """ },
    {"role": "user", "content": f"{context}"}]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=current_question, temperature=0.0) # temperature(0.0) will generate same question and answer
    res = response['choices'][0]['message']['content']
    questions = res.split('\\n\\n')
    return questions

In [ ]:
# Function calling and appending the mca_questions into a single list
mcq=[]
for i in text_chunks:
  context = i
  response = get_mca_questions(context)
  mcq.extend(response)


In [ ]:
# Printing the mca questions
for i in mcq:
  print(i)

Question 1: Who was the last powerful Mughal ruler?
A) Bahadur Shah Zafar
B) Aurangzeb
C) Alivardi Khan
D) Sirajuddaulah
Answers: B) Aurangzeb and A) Bahadur Shah Zafar

Question 2: Which European power had already established its presence in the western coast of India before the arrival of the British?
A) Dutch
B) French
C) Portuguese
D) Spanish
Answers: C) Portuguese and A) Dutch

Question 3: What was the main reason for the Battle of Plassey?
A) To establish control over the Company's fort in Calcutta
B) To secure trade concessions and other privileges from the nawab
C) To defeat the forces led by Mir Jafar
D) To expand the Company's settlements and fortifications
Answers: B) To secure trade concessions and other privileges from the nawab and C) To defeat the forces led by Mir Jafar

Question 4: What did the Company become after being appointed as the Diwan of the provinces of Bengal?
A) Nawabs
B) Rulers
C) Merchants
D) Diwans
Answers: D) Diwans and B) Rulers

Question 5: What did i

# Using Google PaLM PAI by passing whole passage into text-bison model, Generating mca questions

In [ ]:
def get_mca_questions_1(passage):
    prompt = f"""You are an AI assitant tasked with generating multiple-choice questions (MCQs) by reading the given context.
    Generate the Multiple Choice question as per Given Example format.
    Each question must have two answers.
    Example:
    Question 1: Which of the following planets in our solar system have rings?
    A) Mercury
    B) Venus
    C) Saturn
    D) Mars
    Correct Answers: C) Saturn and D) Mars
    context : {passage}, Multiple-choice-questions: """
    completion = palm.generate_text(
    model= "models/text-bison-001",
    prompt=prompt,
    temperature=0.0,
    max_output_tokens=1024,
)

    res = completion.candidates[0]['output']
    question = res.split('\n')
    return question

In [ ]:
res = get_mca_questions_1(content)
res

['1. The East India Company was transformed from a trading company to a territorial colonial power because:',
 '(a) it was able to defeat the Marathas.',
 '(b) it was able to defeat the Mughals.',
 '(c) it was able to defeat the Sikhs.',
 '(d) all of the above.',
 '2. The Doctrine of Lapse was implemented by:',
 '(a) Warren Hastings.',
 '(b) Lord Dalhousie.',
 '(c) Lord Cornwallis.',
 '(d) Lord Wellesley.',
 '3. The Company army was composed of:',
 '(a) cavalry and infantry.',
 '(b) cavalry and paidal soldiers.',
 '(c) cavalry and sepoys.',
 '(d) all of the above.',
 '4. The British began to develop a uniform military culture in the:',
 '(a) early nineteenth century.',
 '(b) late eighteenth century.',
 '(c) early twentieth century.',
 '(d) late nineteenth century.',
 '5. The East India Company came to exercise direct rule over about 63 per cent of the territory and 78 per cent of the population of the Indian subcontinent by:',
 '(a) 1857.',
 '(b) 1858.',
 '(c) 1859.',
 '(d) 1860.']

# Using Google PaLM PAI by passing whole passage into chat-bison model, Generating mca questions

In [ ]:
def get_mca_questions_2(passage):
    prompt = f"""You are an AI assitant tasked with generating multiple-choice questions (MCQs) by reading the given context.
    Generate the Multiple Choice question as per Given Example format.
    Each question must have two answers.
    Example:
    Question 1: Which of the following planets in our solar system have rings?
    A) Mercury
    B) Venus
    C) Saturn
    D) Mars
    Correct Answers: C) Saturn and D) Mars
    context : {passage}, Multiple-choice-questions: """
    completion = palm.chat(
    model= "models/chat-bison-001",
    messages=prompt,
    temperature =0.0,
)

    return (completion)

In [ ]:
for i in text_chunks:
  context=i
  response = get_mca_questions_2(context)
  print(response.last)

1. The East India Company was granted a charter by Queen Elizabeth I in:
    * 1600
    * 1651
    * 1757
    * 1765

2. The East India Company's first factory in India was set up in:
    * Surat
    * Masulipatnam
    * Hugli
    * Madras

3. The Battle of Plassey was fought between the East India Company and the Nawab of Bengal in:
    * 1756
    * 1757
    * 1764
    * 1765

4. The Battle of Plassey was a turning point in the history of British India because it:
    * marked the beginning of British rule in India
    * led to the establishment of the East India Company as a major power in India
    * gave the East India Company control of the province of Bengal
    * all of the above

5. The East India Company became the Diwan of the provinces of Bengal in:
    * 1757
    * 1764
    * 1765
    * 1772

6. The Diwani allowed the East India Company to:
    * collect taxes in Bengal
    * administer the provinces of Bengal
    * both of the above
    * none of the above

7. The assumpti

# Conclusion:

* Automatically generate objective queries with multiple valid responses from a designated chapter within a specific subject. The creation of multiple-choice questions relies on a sophisticated language model's capacity to comprehend the content and produce inquiries. I explored three different approaches to question generation:

* OpenAI's GPT-3.5 Turbo model
* Google's Palm PAI with text-bison
* Google's Chat PAI with text-bison

* Among these three models, the OpenAI GPT-3.5 Turbo model consistently delivered the most impressive results.